In [54]:
import numpy as np
import pandas as pd
import datetime
################## INSERT CCY You want to RUN
ccy = "EURUSD"
edi_location = r"C:\Users\edgil\Documents\Masters\dissertation\data\EDI.csv"
esi_location = r"C:\Users\edgil\Documents\Masters\dissertation\data\ESI.csv"
EDI = pd.read_csv(edi_location)
ESI = pd.read_csv(esi_location)
EDI['Date'] =  pd.to_datetime(EDI['Date'], format= '%d/%m/%Y')
ESI['Date'] =  pd.to_datetime(ESI['Date'], format= '%d/%m/%Y')
ccy1 = ccy[:3]
ccy2 = ccy[3:]
EDI["EDI_"+ ccy1 + ccy2 + "_spread"] = EDI["EDI_" +ccy1 + "_normalised"] - EDI["EDI_" +ccy2 + "_normalised"]
ESI["ESI_"+ ccy1 + ccy2 + "_spread"] = ESI["ESI_" +ccy1 + "_normalised"] - ESI["ESI_" + ccy2 + "_normalised"]
EDI = EDI[['Date',"EDI_"+ccy1 + ccy2 + "_spread", "EDI_G10_normalised", "EDI_Global_normalised"]]
ESI = ESI[['Date',"ESI_"+ccy1 + ccy2 + "_spread", "ESI_G10_normalised", "ESI_Global_normalised"]]
signal_location = r"C:\Users\edgil\Documents\Masters\dissertation\data\best_results\Signals.csv"
signals = pd.read_csv(signal_location)
signals['Date'] =  pd.to_datetime(signals['Date'], format= '%d/%m/%Y %H:%M')
signals['DDMMYY'] = signals['Date'].dt.date # signals['Date'].apply(get_ddmmyy)
#signals['DDMMYY'] = pd.to_datetime(signals['DDMMYY'])
trend_estimation = signals.groupby('DDMMYY').sum().reset_index(drop = False)
# i.e. the average of the ccy kevel across each hour
trend_estimation['CCY'] = signals.groupby('DDMMYY').mean().reset_index(drop = False)['CCY']
# overwriting the dte arg as its easier to understand
trend_estimation['Date'] = trend_estimation['DDMMYY']
trend_estimation = pd.merge(trend_estimation,EDI,how="left" ,on= "Date").fillna(0)
trend_estimation = pd.merge(trend_estimation,ESI,how="left" ,on= "Date").fillna(0)

trend_estimation.to_csv()

',DDMMYY,CCY,logret,Linear,SVM_erf,RF_erf,LSTM1_erf,LSTM2_erf,Date,EDI_EURUSD_spread,EDI_G10_normalised,EDI_Global_normalised,ESI_EURUSD_spread,ESI_G10_normalised,ESI_Global_normalised\n0,2000-02-29,0.9651250000000001,-0.007180711000000001,0.0,0.0,0.0,0.0,0.0,2000-02-29,0.0,0.0,0.0,0.0,0.0,0.0\n1,2000-03-01,0.9690333333333334,0.009391678,0.00239772,0.0,0.001692124,0.0,0.0,2000-03-01,0.0,0.0,0.0,0.0,0.0,0.0\n2,2000-03-02,0.9709499999999998,-0.008148154,-0.006683854999999999,0.0,0.00028428899999999974,-0.004147777,0.00546973,2000-03-02,0.0,0.0,0.0,0.0,0.0,0.0\n3,2000-03-03,0.9639083333333334,-0.007171469,0.0032903100000000003,0.0,-0.000496662,-0.009069793999999999,-0.006535307,2000-03-03,0.0,0.0,0.0,0.0,0.0,0.0\n4,2000-03-06,0.9597458333333336,-5.421010862427522e-19,-0.0025651870000000005,0.0,-0.0005809830000000002,0.00013340600000000128,-0.0014578790000000002,2000-03-06,0.0,0.0,0.0,0.0,0.0,0.0\n5,2000-03-07,0.9581791666666666,-0.00046949600000000024,0.000685915,1.200000000000009e-06,-0.

In [46]:
from matplotlib import pyplot as plt

print(trend_estimation)

          DDMMYY       CCY        logret    Linear       SVM_erf    RF_erf  \
0     2000-02-29  0.965125 -7.180711e-03  0.000000  0.000000e+00  0.000000   
1     2000-03-01  0.969033  9.391678e-03  0.002398  0.000000e+00  0.001692   
2     2000-03-02  0.970950 -8.148154e-03 -0.006684  0.000000e+00  0.000284   
3     2000-03-03  0.963908 -7.171469e-03  0.003290  0.000000e+00 -0.000497   
4     2000-03-06  0.959746 -5.421011e-19 -0.002565  0.000000e+00 -0.000581   
5     2000-03-07  0.958179 -4.694960e-04  0.000686  1.187800e-06 -0.000374   
6     2000-03-08  0.958637  2.865705e-03  0.000475 -4.265186e-04  0.001282   
7     2000-03-09  0.964350  6.224088e-03  0.002291 -1.602793e-03  0.002876   
8     2000-03-10  0.965183 -3.418452e-03 -0.002555  9.653594e-04 -0.001744   
9     2000-03-13  0.966529 -5.188500e-05 -0.004902 -4.688070e-04 -0.000151   
10    2000-03-14  0.964388  5.071949e-03  0.000278 -2.390320e-05  0.002150   
11    2000-03-15  0.967629 -2.118791e-03 -0.000777 -2.646229e-04

In [38]:
x = signals['Date'].loc[1]
#datetime.datetime.strptime(x,"%d/%m/%y HH:MM")
type(x)
#datetime.datetime.strftime(x,"%d/%m/%YY")

str